# Introduction 

Automatic differentiation (AD) is a set of techniques to numerically evaluate the derivative of a function specified by a computer program. In mathematics and computer fields, a large number of numerical methods require derivatives to be calculated(Margossian, 2019). For example, in machine learning, gradient plays an essential role. Neural networks are able to gradually increase accuracy with every training session through the process of gradient descent(Wang, 2019).  In physics, differentiation is needed to quantify the rate of change in the equations of motion. However, for many problems, calculating derivatives analytically is both time consuming and computationally impractical. Automatic differentiation can gives exact answers in constant time(Saroufim, 2019), which is a powerful tool to automate the calculation of derivatives, especially when differentiating complex algorithms and mathematical functions(Margossian, 2019).  

There are three main methods of calculating derivatives: numerical differentiation, symbolic differentiation, and automatic differentiation. Specifically, symbolic differentiation involves computing exact expressions for the derivatives in terms of the function variables by representing the expression as a tree and manipulating it using the rules of differentiation. The derivatives can quickly become very complicated for complex functions and higher-order derivatives. Thus, for complicated functions, symbolic differentiation is not easily generalizable and becomes computationally inefficient due to redundant evaluations of different components of the function . Numerical differentiation aims to approximate derivatives through finite differentiating, but the solution accuracy is greatly affected by the truncation and round-off errors associated with finite difference formulas. Automatic differentiation is more computationally efficient and generalizable than symbolic differentiation and can find exact derivatives as opposed to numerical differentiation.

In this library, we will describe the mathematical background and concepts in the Background section, explain the software organization, and discuss the implementation of the forward mode of automatic differentiation.



# Background

Automatic differentiation assumes that we are working with a differentiable function composed of a finite number of elementary functions and operations with known symbolic derivatives. The table below shows some examples of elementary functions and their respective derivatives:

| Elementary Function    | Derivative              |
| :--------------------: | :----------------------:|
| $x^3$                  | $3 x^{2}$               | 
| $e^x$                  | $e^x$                   |
| $\sin(x)$              | $\cos(x)$               |
| $\ln(x)$               | $\frac{1}{x}$           |

Given a list of elementary functions and their corresponding derivatives, the automatic differentiation process involves the evaluations of the derivatives of complex compositions of these elementary functions through repeated applications of the chain rule:

$ \frac{\partial}{\partial x}\left[f_1 \circ (f_2 \circ \ldots (f_{n-1} \circ f_n)) \right] = 
\frac{\partial f_1}{\partial f_2} \frac{\partial f_2}{\partial f_3} \ldots \frac{\partial f_{n-1}}{\partial f_n}\frac{\partial f_n}{\partial x}$

This process can be applied to the evaluation of partial derivatives as well thus allowing for computing derivatives of multivariate and vector-valued functions.

## Computational Graph

The forward mode automatic differentiation process described above can be visualized in a computational graph, a directed graph with each node corresponding to the result of an elementary operation.

For example, consider the simple problem of evaluating the following function and its derivative at $x=2$:
$$
f(x) = x^3 +2x^2
$$
The evaluation of this function can be represented by the evaluation trace and computational graph below where the numbers in orange are the function values and the numbers in blue are the derivatives evaluated after applying each elementary operation:

<img src="https://raw.githubusercontent.com/anita76/playground/master/src/ex_comp_graph.png" width="75%" />

| Trace       | Elementary Function     | Value   | Derivative                | Derivative Value      |
| :---------: | :----------------------:| :------:| :------------------------:| :--------------------:|
| $v_1$       | $x$                     | $2$     | $\dot{x}$                 | $1$                   |
| $v_2$       | $v_1^2$                 | $4$     | $2v_1 \dot{v}_1$          | $4$                   |
| $v_3$       | $v_1^3$                 | $8$     | $3v_1^2 \dot{v}_1$        | $12$                  |
| $v_4$       | $2v_2$                  | $8$     | $2 \dot{v}_2$             | $8$                   |
| $v_5$       | $v_3 + v_4$             | $16$    | $ \dot{v}_3 + \dot{v}_4$  | $20$                  |



# How to Use West Coast AD

### Installation with GitHub
1. Clone the package repository:

    `git clone https://github.com/West-Coast-Differentiators/cs107-FinalProject.git`

2. If you do not have Anaconda installed, follow the instructions on their [website](https://deepnote.com/project/07e48b43-3d96-4960-83bf-911527fb55ea?cellId=00002-de6540a6-d222-4d8b-b063-a392b8a9b7d4#/milestone2-2.ipynb) to install it.
3. Create a new conda environment:

    `conda create --name <env_name> python=3.8` 

4. Activate your virtual environment:

    `conda activate <env_name>`

5. Navigate to the base repository:

    `cd cs107-FinalProject`

6. Install the package and its requirements:

    `pip install ./`

7. You may check that the installation was successful by running the package tests:

    `python -m unittest discover -s WestCoastAD/test -p '*_test.py'`

### Using the Package

 Having installed the package, users can evaluate derivatives of functions written in terms of WestCoastAD variables. For instance, the derivatives of $f(x) = x^3+ 2x^2$ and $f(x) = sin(x) + cos(x)$ can be evaluated as follows:


In [ ]:
# 1) import WestCoastAD
>>> from WestCoastAD import Variable
# 2) define the variables of your function as WestCoastAD variables 
#    with the value at which you want to evaluate the derivative and 
#    a derivative seed value
>>> x = Variable(value=2, derivative_seed=1) 
# 3) define your function in terms of the WestCoastAD variable objects
>>> f = x**3 + 2* x**2
# 4) access the computed derivative and the value of the function 
>>> print(f.value)
16
>>> print(f.derivative)
20
# 5) For other mathematical operations that are not part of the standard python library,
#    the function could be called with numpy or directly on the variable itself.
#    For a full list of supported elementary functions, refer to the software implementation
#    section of this document.
>>> import numpy as np
>>> x = Variable(value=np.pi/2, derivative_seed=1)
# 6) As an example, below we directly call sin on the variable and use numpy to call cos
>>> f = x.sin() + np.cos(x)
>>> print(f.value)
1.0
>>> print(f.derivative)
-0.9999999999

# Software Organization
## Directory Structure
```bash
cs107-FinalProject  
             setup.py
             .travis.yml 
             README.MD
             docs/
                             __init__.py
                             milestone1.ipynb
                             milestone2.ipynb   
                             milestone2A.ipynb
                             milestone2_progress.ipynb  
                             ...
             WestCoastAD/  
                             __init__.py
                             src/    
                                     __init__.py
                                     forward_mode_AD.py
                                     gradient_descent_optimizer.py
                                     ...

                             tests/ 
                                     __init__.py
                                     forward_mode_AD_test.py 
                                     gradient_descent_optimizer_test.py
                                     ...
                     ...
```
## Modules and Basic Functionality

#### Docs Module
* This module contains the documentation for the package. The documentations include the information about the package including the purpose, the organization of the software, how to use the package and how to install the package.

#### Setup Module
* This module contains configurations to allow users to install our package. Inside the setup.py file, we use the setuptools library for packaging the modules and installing the package dependencies. Our package requires that numpy be installed.

#### Forward Mode Module
* This module includes the implementation of the forward mode of automatic differentiation. The module defines a Variable class that will take the value to be evaluated and the derivative seed value. The derivative and value of functions defined in terms of the Variable class, can be accessed by viewing the derivative and value attributes of this class.

#### Gradient Descent Optimization Module
* This is the extension module that will be added in a future release to include our extension for the AD package. The future section of the documentation below, describes our AD extension.

#### Test Modules
* This module contains all the tests of our implementation including unit tests and acceptance tests. We used Python's unittest to define the tests and check our coverage with CodeCov. Having successfully installed the package following the instruction in the "How to Use WestCoastAD" section of this documentation, the tests can be run using the following command:
`python -m unittest discover -s WestCoastAD/test -p '*_test.py'`. Alternatively, you can run individual test files using: `python WestCoastAD/test/<TEST_FILE_NAME>.py`

## Packaging and Build Pipeline

#### Packaging
* We packaged our module with setuptools following standard python packaging protocols as described in the following tutorial: [https://python-packaging-tutorial.readthedocs.io/en/latest/setup_py.html](https://python-packaging-tutorial.readthedocs.io/en/latest/setup_py.html)

#### Distribution
* WestCoastAD is currently distributed via Github. In the "How to Use WestCoast AD" section of this document, there is information about where to get the package, how to install a conda environment and how to install the package itself.

#### Travis CI
* Travis CI allows for automatic building and testing of commits that are pushed to remote. It requires that all pull request be tested and if all the tests are passed, it will update the build automatically. Travis CI also contains a badge that will inform the user if the package is working or if it is down. The travis.yml file is located in our main directory.

#### CodeCov
* CodeCov provides details about our test coverage. This allows us to identify where tests need to be added and will help us discover possible bugs in the code. CodeCov also contains a badge that will inform the user of the percentage code coverage of our tests.




# Software Implementation

This section covers the class and function reference for `WestCoastAD`.

`Variable` is the base class used to instantiate gradient objects and to perform derivatives.
The instances of the `Variable` class should be used to express the scalar or vector function to be differentiated.

| Class Attributes        | Usage        |
| :----------------------- |:-------------|
| `value`                 | Value of the variable (`int` or `float`). The setter and getter for value are defined using the  `property` decorator.|
| `derivative`            | Derivative of the function (`int` or `float`). A `seed derivative` can be provided at the time of instantiation. This attribute is set and retrieved using a `property` decorator.|

<br>

**Class Methods**

###### `__add__`
Dunder method for overloading the "+" operator. This method computes the value and the derivative of the summation operation on `int`, `float` & `Variable` instances. This method returns a new `Variable` object.

###### `__radd__`
Same method as `__add__` with reversed operands.

###### `__mul__`
Dunder method for overloading the "*" operator. This method computes the value and the derivative following the [product rule](https://en.wikipedia.org/wiki/Product_rule) on `int`, `float` & `Variable` instances. It returns a new `Variable` object.

###### `__rmul__`
Same method as `__mul__` with reversed operands.

###### `__sub__`
Dunder method for overloading the "-" operator. This method computes the value and the derivative of the substraction operation on `int`, `float` & `Variable` instances. It returns a new `Variable` object.

###### `__rsub__`
Same method as `__sub__` with reversed operands.

###### `__truediv__`
Dunder method for overloading the "/" operator. This method computes the value and the derivative following the [quotient rule](https://en.wikipedia.org/wiki/Quotient_rule) on `int`, `float` & `Variable` instances. It returns a new `Variable` object. This method throws a `ZeroDivisionError` if the divisor is 0.

###### `__rtruediv__`
Same method as `__truediv__` with reversed operands.

###### `__pow__`
Dunder method for overloading the "**" operator. This method computes the value and the derivative following the [power rule](https://en.wikipedia.org/wiki/Power_rule) if exponent is `int` or `float`.
It returns a new `Variable` object. This method throws a `ValueError` if:
* a negative valued variable/function is raised to a non-integer power (e.g. $x^{1.2}, x=-2$)
* a zero valued variable/function is raised to a non-positive power (e.g. $x^{-5}, x = 0$)
* a non-positive valued variable/function is raised to the power of another variable/function (e.g. $x^{sin(x)}, x=-1$). This is because the generalized power rule of differentiation takes the logarithm of the base and the logarithm is not defined for non-positive values.

###### `__rpow__`
Dunder method which handles the differentiation of equations of the form $a^{f(x)}$. It returns a new `Variable` object. This method throws a `ValueError` if:
* zero is raised to the power of a variable/function with a non-positive value (e.g. $0^{x}, x=0$). 
* a negative number is used as the base (e.g. $(-2)^x$)

###### `__neg__`
Dunder method for overloading the negation operation. This method computes the value and derivative of the negation operation and returns a new `Variable` object.

###### `__abs__`
Dunder method for overloading the abs function. This method computes the value and derivative of the absolute value of a variable and returns a `Variable` object. It will throw a `ValueError` if given a value of zero since the derivative of absolute value is not defined at zero.

###### `__repr__`
Dunder method for overloading the `repr` function. Returns the object representation of the class `Variable` in the form `Variable(value=value, derivative=derivative)`

###### `log`
This method computes the value and derivative of `log` function and returns a new `Variable` object. A `ValueError` is thrown for non-positive values.

###### `exp`
This method computes the value and derivative of `exp` function and returns a new `Variable` object

###### `sqrt`
Square root is a special case of pow and therefore calls `__pow__` with exponent `1\2`.

###### `sin`

This method computes the value and the derivative of the sine function. It returns a new `Variable` object.

###### `cos`

This method computes the value and the derivative of the cosine function. It returns a new `Variable` object.

###### `tan`

This method computes the value and the derivative of the tangent function. It returns a new `Variable` object. This method will throw a `ValueError` if given a value that is an odd multiple of $\frac{\pi}{2}$

###### `sinh`

This method computes the value and the derivative of the hyperbolic sine function. It returns a new `Variable` object.

###### `cosh`

This method computes the value and the derivative of the hyperbolic cosine function.  It returns a new `Variable` object.

###### `tanh`

This method computes the value and the derivative of the hyperbolic tangent function.  It returns a new `Variable` object.

###### `arcsin`

This method computes the value and the derivative of the arcsin function.  It should be given a value in  (-1,1) for the derivative to be defined, otherwise a `ValueError` will be raised. It returns a new `Variable` object.

###### `arccos`

This method computes the value and the derivative of the arccos function. It should be given a value in  (-1,1) for the derivative to be defined, otherwise a `ValueError` will be raised. It returns a new `Variable` object.

###### `arctan`

This method computes the value and the derivative of the arctan function. It returns a new `Variable` object.


#### External Dependencies
We use `numpy` to evaluate elementary functions which were not available as part of the Python standard library (e.g. sin, cos, etc). We will also be accepting numpy as an input in later versions when we extend the implementation to also compute gradients and Jacobians of multivariable and vector-valued functions.


# Future Features



### Vector valued and multivariable functions 
In addition to using floats and ints as input, we will accept numpy arrays as input to the Variable class. This is to accommodate for the computation of gradients of multivariable functions as well as the Jacobians of vector-valued functions. This will require modifications to the methods defined in the Variable class as well as the implementation of new elementary operations such as matrix multiplication.
### Extention class:  optimization with gradient descent built upon the AD core

Gradient-based optimization is an essential method in the field of machine learning. It provides a simple and iterative algorithm for finding local minima of a real-valued function $F$ numerically. Given an objective function, gradient-based methods make use of the fact that the objective function decreases steepest if one goes in the direction of the negative gradient: $-\nabla F(x)$. The iteration step of the algorithm is defined in terms of a step size parameter $\eta$ as:
$$
x_{i+1} = x_i - \eta\nabla F(x_i) 
$$
Automatic differentiation  provides an efficient way for gradient computation by creating computation graph. We will be implementing the algorithm for gradient descent in a new module under our WestCoastAD package and using our forward_mode_AD module to compute the derivative for each iteration. the `gradient_descent_optimizer.py` file, will implement a function called `optimize_gd` which will take as input the function to be optimized and a list of all the WestCoastAD variables used to define the function. The function will return the optimum values for the variables in the function after performing gradient descent. The solution will be of the same shape as the input variables.  Gradient descent parameters such as the number of iterations and the learning rate can also be optionally passed into the `optimize_gd` function if the user does not want to use the default values for these parameters.




# References
-  Margossian, C. C. (2019). A review of automatic differentiation and its efficient implementation. Wiley Interdisciplinary Reviews: Data Mining and Knowledge Discovery, 9(4), 1–32. https://doi.org/10.1002/widm.1305

- Saroufim, M. (2019, November 13). Automatic Differentiation Step by Step. Retrieved October 15, 2020, from https://marksaroufim.medium.com/automatic-differentiation-step-by-step-24240f97a6e6

- Wang, C. (2019, March 3). Automatic Differentiation, Explained. Retrieved October15, 2020,from https://towardsdatascience.com/automatic-differentiation-explained-	b4ba8e60c2ad

- Automatic Differentiation Background. Retrived October 20, 2020, from https://www.mathworks.com/help/deeplearning/ug/deep-learning-with-automatic-differentiation-in-matlab.html